In [1]:
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pathlib
import cv2

In [2]:
data_dir = pathlib.Path('C:/Users/Blake/Downloads/Shoes')

In [40]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    label_mode = 'categorical',
    image_size = (224, 224),
    batch_size = 32
)

Found 6200 files belonging to 3 classes.
Using 4960 files for training.


In [41]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    label_mode = 'categorical',
    image_size = (224, 224),
    batch_size = 32
)

Found 6200 files belonging to 3 classes.
Using 1240 files for validation.


In [42]:
class_names = train_ds.class_names

In [43]:
resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(include_top = False,
                                                  input_shape = (224, 224, 3),
                                                  pooling = 'avg',
                                                  classes = len(class_names),
                                                  weights = 'imagenet')

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation = 'relu'))
resnet_model.add(Dense(len(class_names), activation = 'softmax'))

In [44]:
resnet_model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [47]:
history = resnet_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10
)

Epoch 1/10
155/155 [==============================] - 420s 3s/step - loss: 2.1219e-04 - accuracy: 1.0000 - val_loss: 1.5201e-04 - val_accuracy: 1.0000
Epoch 2/10
155/155 [==============================] - 399s 3s/step - loss: 1.3301e-04 - accuracy: 1.0000 - val_loss: 1.0193e-04 - val_accuracy: 1.0000
Epoch 3/10
155/155 [==============================] - 399s 3s/step - loss: 9.0528e-05 - accuracy: 1.0000 - val_loss: 7.4192e-05 - val_accuracy: 1.0000
Epoch 4/10
155/155 [==============================] - 404s 3s/step - loss: 6.5171e-05 - accuracy: 1.0000 - val_loss: 5.5275e-05 - val_accuracy: 1.0000
Epoch 5/10
155/155 [==============================] - 427s 3s/step - loss: 4.9455e-05 - accuracy: 1.0000 - val_loss: 4.2684e-05 - val_accuracy: 1.0000
Epoch 6/10
155/155 [==============================] - 424s 3s/step - loss: 3.7941e-05 - accuracy: 1.0000 - val_loss: 3.4160e-05 - val_accuracy: 1.0000
Epoch 7/10
155/155 [==============================] - 425s 3s/step - loss: 3.0361e-05 - accura

In [128]:
image = cv2.imread('C:/Users/Blake/Downloads/Demo/A1.JPG')
image_resized = cv2.resize(image, (224, 224))
image = np.expand_dims(image_resized, axis = 0)

pred = resnet_model.predict(image)
output_class = class_names[np.argmax(pred)]

print('Predicted shoe:', output_class)

1/1 [==============================] - 0s 108ms/step
Predicted shoe: Under Armour Ripple 2.0 Sneaker


In [108]:
resnet_model.save('ResNet50_V3')

INFO:tensorflow:Assets written to: ResNet50_V3\assets


INFO:tensorflow:Assets written to: ResNet50_V3\assets
